In [1]:
import numpy as np
import pandas as pd

In [2]:
df_train = pd.read_csv('../dataset/df_train_churn.csv')
df_train['created_at'] = pd.to_datetime(df_train['created_at'], format="%Y-%m-%d")
df_train['shipment_date_limit'] = pd.to_datetime(df_train['shipment_date_limit'], format="%Y-%m-%d").dt.tz_localize(None).dt.normalize()

In [4]:
df_train['time_purchase_delivery'] = df_train.apply(lambda x: (x['shipment_date_limit'] - x['created_at']).days, axis=1)
df_train['delivery_percent_total_amount'] = df_train.apply(lambda x: (x['shipment_fee']*100)/x['total_amount'], axis=1)
df_train['is_shipment_fee'] = np.where(df_train['shipment_fee'] > 0, 1, 0)
df_train['is_promo_code'] = np.where(~df_train['promo_code'].isna(), 1, 0)

df_quant_prod = df_train.groupby(['created_at', 'customer_id', 'booking_id']).agg(quant_prod_purchased=('quantity', 'sum')).reset_index()
df_train_features = pd.merge(df_train, df_quant_prod, on=['created_at', 'customer_id', 'booking_id'])

In [5]:
df_train_features = df_train_features.drop(columns=['booking_id', 'session_id', 'payment_status', 'promo_code',
                                                    'shipment_date_limit', 'shipment_location_lat', 'shipment_location_long',  
                                                    'product_id', 'quantity', 'item_price', 
                                                    'next_date', 'lag_date'])
df_train_features.drop_duplicates(inplace=True)
df_train_features

,created_at,customer_id,payment_method,promo_amount,shipment_fee,total_amount,churn,time_purchase_delivery,delivery_percent_total_amount,is_shipment_fee,is_promo_code,quant_prod_purchased
0,2018-07-29,5868,Debit Card,1415,10000,199832,1,5,5.004204,1,1,1
1,2018-07-30,4774,Credit Card,0,10000,155526,0,4,6.429793,1,0,1
2,2018-09-15,4774,OVO,0,10000,550696,0,3,1.815884,1,0,4
3,2018-11-01,4774,Credit Card,0,0,271012,0,4,0.000000,0,0,1
4,2018-12-18,4774,Credit Card,0,0,198753,0,5,0.000000,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1201138,2020-11-12,25121,Debit Card,0,10000,287688,1,6,3.475988,1,0,1
1201139,2021-12-09,34829,OVO,5010,10000,359203,1,3,2.783941,1,1,1
1201140,2021-12-11,99645,Credit Card,2181,5000,1158159,1,1,0.431720,1,1,4
1201141,2019-12-18,87319,Credit Card,4503,0,184143,1,5,0.000000,0,1,1


In [6]:
df_train_features.to_csv('../dataset/df_train_feature_churn.csv', index=False)